In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

In [2]:
from tensorflow.keras.models import load_model
model = load_model('Final_Assignment_2_model.h5', compile=False)

In [3]:
import tensorflow as tf
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

In [8]:
video_capture = cv2.VideoCapture(1)

In [9]:
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    crop_image = []
    for (x, y, w, h) in faces:
        y = y-5
        x = x-5
        h = h+10
        w = w+10
        crop_image = vid[y:y+h, x:x+w]
        resize_img = cv2.resize(crop_image, (100, 100))
        resize_img = cv2.cvtColor(resize_img, cv2.COLOR_BGR2RGB)
        predictions  = model.predict(np.array([resize_img]), verbose = 0)
        binary_prediction = (predictions > 0.5).astype(int)
        if binary_prediction == 0:
            cv2.putText(vid, "Mask", (x+10, y+h+10), 1, 1.0, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
        else:
            cv2.putText(vid, "No Mask", (x+10, y+h+10), 1, 1.0, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 0, 255), 4)
        
    return faces

In [10]:
while True:

    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully

    faces = detect_bounding_box(
        video_frame
    )  # apply the function we created to the video frame

    cv2.imshow(
        "My Face Detection Project", video_frame
    )  # display the processed frame in a window named "My Face Detection Project"

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()